In [32]:
'''
We explore the Pennylane model that lambeq provides. 

'''

# import things 
import torch 
import random
import numpy as np

SEED = 12
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [33]:
'''
It seems that the sentence similarity task has already been done before with the datasets of interest. 

'''

# load data 

def read_data(filename):
    labels, sentences = [], []
    with open(filename) as f:
        for line in f:
            line = line.split(',')
            labels.append(int(line[2]))
            sentences.append((line[0], line[1]))
    return labels, sentences

train_labels, train_data = read_data('../datasets/mc_pair_train_data.csv')
dev_labels, dev_data = read_data('../datasets/mc_pair_dev_data.csv')
test_labels, test_data = read_data('../datasets/mc_pair_test_data.csv')


In [34]:
# create diagrams 

# take pairs apart for now, repair them later! 

train_data_l, train_data_r = zip(*train_data)
train_data_unpaired = list(train_data_l) + list(train_data_r)
dev_data_l, dev_data_r = zip(*dev_data)
dev_data_unpaired = list(dev_data_l) + list(dev_data_r)
test_data_l, test_data_r = zip(*test_data)
test_data_unpaired = list(test_data_l) + list(test_data_r)
from lambeq import BobcatParser

reader = BobcatParser(verbose='text')

raw_train_diagrams = reader.sentences2diagrams(train_data_unpaired)
raw_dev_diagrams = reader.sentences2diagrams(dev_data_unpaired)
raw_test_diagrams = reader.sentences2diagrams(test_data_unpaired)

Tagging sentences.
Parsing tagged sentences.
Turning parse trees to diagrams.
Tagging sentences.
Parsing tagged sentences.
Turning parse trees to diagrams.
Tagging sentences.
Parsing tagged sentences.
Turning parse trees to diagrams.


In [35]:
# simplify diagrams by removing cups 

from lambeq import RemoveCupsRewriter

remove_cups = RemoveCupsRewriter()

train_diagrams = [remove_cups(diagram) for diagram in raw_train_diagrams]
dev_diagrams = [remove_cups(diagram) for diagram in raw_dev_diagrams]
test_diagrams = [remove_cups(diagram) for diagram in raw_test_diagrams]

In [36]:
'''
Create circuits 
'''

from lambeq import AtomicType, IQPAnsatz

ansatz = IQPAnsatz({AtomicType.NOUN: 1, AtomicType.SENTENCE: 1},
                   n_layers=1, n_single_qubit_params=3)

train_circuits = [ansatz(diagram) for diagram in train_diagrams]
dev_circuits =  [ansatz(diagram) for diagram in dev_diagrams]
test_circuits = [ansatz(diagram) for diagram in test_diagrams]


In [37]:
BATCH_SIZE = 50
EPOCHS = 100
SEED = 2

In [38]:
from torch import nn
from lambeq import PennyLaneModel

# inherit from PennyLaneModel to use the PennyLane circuit evaluation
class XORSentenceModel(PennyLaneModel):
    def __init__(self, **kwargs):
        PennyLaneModel.__init__(self, **kwargs)

        self.xor_net = nn.Sequential(nn.Linear(4, 10),
                                     nn.ReLU(),
                                     nn.Linear(10, 1),
                                     nn.Sigmoid())

    def forward(self, diagram_pairs):
        first_d, second_d = zip(*diagram_pairs)
        # evaluate each circuit and concatenate the results
        evaluated_pairs = torch.cat((self.get_diagram_output(first_d),
                                     self.get_diagram_output(second_d)),
                                    dim=1)
        evaluated_pairs = 2 * (evaluated_pairs - 0.5)
        # pass the concatenated results through a simple neural network
        return self.xor_net(evaluated_pairs)

In [39]:
'''
Create paired data
'''

def make_pair_data(diagrams):
    pair_diags = list(zip(diagrams[:len(diagrams)//2], diagrams[len(diagrams)//2:]))
    return pair_diags

train_pair_circuits = make_pair_data(train_circuits)
dev_pair_circuits = make_pair_data(dev_circuits)
test_pair_circuits = make_pair_data(test_circuits)


In [40]:
from lambeq import Dataset
# from lambeq import PennyLaneModel



all_pair_circuits = (train_pair_circuits +
                     dev_pair_circuits +
                     test_pair_circuits)
a, b = zip(*all_pair_circuits)

# initialise our model by passing in the diagrams, so that we have trainable parameters for each token
model = XORSentenceModel.from_diagrams(a + b, probabilities=True, normalize=True)
model.initialise_weights()
model = model.double()

# initialise datasets and optimizers as in PyTorch
train_pair_dataset = Dataset(train_pair_circuits,
                             train_labels,
                             batch_size=BATCH_SIZE)

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [41]:
def accuracy(circs, labels):
    predicted = model(circs)
    return (torch.round(torch.flatten(predicted)) ==
            torch.DoubleTensor(labels)).sum().item()/len(circs)

In [46]:
best = {'acc': 0, 'epoch': 0}

for i in range(EPOCHS):
    epoch_loss = 0
    for circuits, labels in train_pair_dataset:
        optimizer.zero_grad()
        predicted = model(circuits)
        # use BCELoss as our outputs are probabilities, and labels are binary
        loss = torch.nn.functional.binary_cross_entropy(
            torch.flatten(predicted), torch.DoubleTensor(labels))
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

    # evaluate on dev set every 5 epochs
    # save the model if it's the best so far
    # stop training if the model hasn't improved for 10 epochs
    if i % 5 == 0:
        dev_acc = accuracy(dev_pair_circuits, dev_labels)

        print('Epoch: {}'.format(i))
        print('Train loss: {}'.format(epoch_loss))
        print('Dev acc: {}'.format(dev_acc))

        if dev_acc > best['acc']:
            best['acc'] = dev_acc
            best['epoch'] = i
            model.save('xor_model.lt')
        # elif i - best['epoch'] >= 10:
        #     print('Early stopping')
        #     break

# load the best performing iteration of the model on the dev set
if best['acc'] > accuracy(dev_pair_circuits, dev_labels):
    model.load('xor_model.lt')
    model = model.double()

Epoch: 0
Train loss: 4.16150390641149
Dev acc: 0.515
Epoch: 5
Train loss: 4.129963023137189
Dev acc: 0.495
Epoch: 10
Train loss: 4.162349233298943
Dev acc: 0.535
Epoch: 15
Train loss: 4.084082146218643
Dev acc: 0.52
Epoch: 20
Train loss: 4.059490078564071
Dev acc: 0.485
Epoch: 25
Train loss: 4.132734674241028
Dev acc: 0.5
Epoch: 30
Train loss: 3.9851123160285646
Dev acc: 0.53
Epoch: 35
Train loss: 3.972572400297392
Dev acc: 0.495
Epoch: 40
Train loss: 4.042745903227465
Dev acc: 0.505
Epoch: 45
Train loss: 3.9525597265521024
Dev acc: 0.505
Epoch: 50
Train loss: 4.0093337579081965
Dev acc: 0.53
Epoch: 55
Train loss: 4.110671271610174
Dev acc: 0.525
Epoch: 60
Train loss: 4.010737598836101
Dev acc: 0.5
Epoch: 65
Train loss: 4.012300672298847
Dev acc: 0.535
Epoch: 70
Train loss: 3.9552167669384435
Dev acc: 0.485
Epoch: 75
Train loss: 3.967403234137479
Dev acc: 0.5
Epoch: 80
Train loss: 3.9566375532049483
Dev acc: 0.525
Epoch: 85
Train loss: 3.962031198242382
Dev acc: 0.49
Epoch: 90
Train lo

In [43]:
print('Final test accuracy: {}'.format(accuracy(test_pair_circuits,
                                                test_labels)))

Final test accuracy: 0.545


In [44]:
xor_labels = [[1, 0, 1, 0], [0, 1, 0, 1], [1, 0, 0, 1], [0, 1, 1, 0]]
# the first two entries correspond to the same label for both sentences,
# the last two to different labels
xor_tensors = torch.tensor(xor_labels).double()

model.xor_net(xor_tensors).detach().numpy()

array([[0.46725587],
       [0.4959365 ],
       [0.46433353],
       [0.50692673]])

In [45]:
# cooking sentence
print(test_data[1][0])

p_circ = test_pair_circuits[0][0].to_pennylane(probabilities=True)
symbol_weight_map = dict(zip(model.symbols, model.weights))
p_circ.initialise_concrete_params(symbol_weight_map)
unnorm = p_circ.eval().detach().numpy()

print(unnorm / np.sum(unnorm))

skillful person prepares software .
[0.30974691 0.69025309]
